In [2]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# Live
## Navigators

In [3]:
'''Navigator Options               StarBETa LIVE'''
url_star = 'https://www.starbet.rs/Live'
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True

star = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
star.get(url_star)
star.maximize_window()

In [4]:
'''Navigator Options               MAXBET LIVE'''
url_max = 'https://www.maxbet.rs/ibet-web-client/#/home/overview#top'
options = Options()
options.headless = False #for not opening the navigator


maxb = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
maxb.get(url_max)
maxb.maximize_window()

In [20]:
'''Navigator Options               ADMIRAL LIVE'''
url_adm = 'https://admiralbet.rs/sport-live'
options = Options()
options.headless = False #for not opening the navigator


adm = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
adm.get(url_adm)
adm.set_window_size(1920, 1080)
time.sleep(4)

adm.switch_to.frame(adm.find_element(By.XPATH, '//*[@id="sportLiveIframe"]'))
fudbal = adm.find_element(By.XPATH, '/html/body/app-root/app-betting-live/div[2]/section[2]/div/app-sport-groups-container/app-sport-group[1]/section/div')
fudbal.click()

In [6]:
'''Navigator Options               SoccerBet LIVE'''

url_soc = 'https://soccerbet.rs/#live/overview'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
soc = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
soc.get(url_soc)
soc.maximize_window()
cancel = soc.find_element(By.XPATH, '//*[@id="promotionPopupModal"]/div/div/div/div/input')

## Functions

In [7]:
def starbet_live():
    df_star = pd.DataFrame(columns=['teams', '1_star','X_star', '2_star', 'goals_star', 'under_star', 'over_star' ])
    for j in range(65):
        try:
            elemento = star.find_element(By.XPATH, f'/html/body/form/div[4]/div/div/div/div[2]/div/div[1]/div[3]/div/table[1]/tbody/tr[{j}]')
            element = star.find_element(By.XPATH, f'/html/body/form/div[4]/div/div/div/div[2]/div/div[1]/div[3]/div/table[1]/tbody/tr[{j}]').text.split('\n')
            
            teams_star = element[1]+'  '+element[2]

            star_1 = element[7].replace('-','1')
            star_X = element[8].replace('-','1')
            star_2 = element[9].replace('-','1')

            goals_star = element[11]
            under_star = element[10].replace('-','1')
            over_star = element[12].replace('-','1')

            #print(element[1]+'  '+element[2], '|', j)
            df_star.loc[len(df_star.index)]  = [teams_star, star_1, star_X, star_2, goals_star, under_star, over_star]
            if j in [15,25,35,45,55,65]:
                elemento.location_once_scrolled_into_view
        except:
            pass
    for k in [75,65,55,45,35,25,15,5,3,2,1]:
        try:
            elemento = star.find_element(By.XPATH, f'/html/body/form/div[4]/div/div/div/div[2]/div/div[1]/div[3]/div/table[1]/tbody/tr[{k}]')
            elemento.location_once_scrolled_into_view
        except:
            pass

    df_star.to_csv('CSV\df_star.csv')

In [8]:
def maxbet_live():
    df_max = pd.DataFrame(columns=['teams', '1_max', 'X_max', '2_max', 'goals_max', 'under_max', 'over_max'])
    for i in tqdm(range(80)):
        try:
            elemento = maxb.find_element(By.XPATH, f'/html/body/div[1]/div[5]/div/div[3]/div[2]/div/div/div/div[1]/live-matches-overview-details/div[{i}]/div').text.split('\n')
            
            teams = elemento[2]+' '+elemento[3]

            maxb_1 = elemento[5].replace('-','1')
            maxb_2 = elemento[6].replace('-','1')
            maxb_3 = elemento[7].replace('-','1')

            goals_max = elemento[8].replace('-','1')
            under_max = elemento[9].replace('-','1')
            over_max = elemento[10].replace('-','1')

            df_max.loc[len(df_max.index)] = [teams, maxb_1, maxb_2,maxb_3, goals_max, under_max, over_max]
            #print(elemento[2:4], i)
        except:
            pass
    
    df_max.to_csv('CSV\df_max.csv')
    return df_max

In [9]:
def admiral_live():

    df_adm = pd.DataFrame(columns=['teams', '1_adm', 'X_adm', '2_adm', 'goals_adm', 'under_adm', 'over_adm'])
    for k in range(40):
        for j in range(6):
            try:
                elemento = adm.find_element(By.XPATH, f'/html/body/app-root/app-betting-live/div[2]/section[2]/div/app-sport-groups-container/app-sport-group[1]/section/app-competition-group[{k}]/section/div[2]/app-event-live[{j}]').text.split('\n')
                
                teams = elemento[2]+' '+ elemento[3]

                adm_1 = elemento[8].replace('-','100') #After POWERQUERY in excel all 100's will become 1
                adm_X = elemento[9].replace('-','100')
                adm_2 = elemento[10].replace('-','100')
                
                goals_adm = elemento[20].replace('-','0')
                
                under = elemento[19].replace('-','100')
                over = elemento[21].replace('-','100')
                df_adm.loc[len(df_adm.index)] = [teams, adm_1, adm_X, adm_2, goals_adm, under, over]
                #print(teams,'|', adm_1, adm_X, adm_2,'    |  ', goals_adm, under ,over)
            except:
                pass
    #cols = ['1_adm', 'X_adm', '2_adm',  'under_adm','goals_adm','over_adm']
    #for c in cols:
    #    df_adm[c]= df_adm[c].astype('float64')

    #df_adm = df_adm[df_adm.goals_adm<5]
    df_adm.to_csv('CSV\\df_adm.csv')
    return df_adm
    

In [10]:
def soccerbet_live():
    df_socc = pd.DataFrame(columns=['teams', '1_socc', 'X_socc', '2_socc', 'goals_socc', 'under_socc', 'over_socc'])
    for i in range(60):
        try:
            element = soc.find_element(By.XPATH, f'//*[@id="result-tables"]/div[2]/div[1]/table/tbody/tr[{i}]').text.split('\n')
            teams = element[1][4:].replace(':','')
            teams = re.sub('\d+','',teams)

            socc_1 = element[2]
            socc_X = element[3]
            socc_2 = element[4]

            goals_socc= element[10][:3]
            under_socc= element[8]
            over_socc=element[9]

            df_socc.loc[len(df_socc.index)] = [teams, socc_1, socc_X,socc_2, goals_socc, under_socc, over_socc]

            #print(teams, socc_1, socc_X, socc_2, '|',under_socc, over_socc)


        except:
            pass
    df_socc.to_csv('CSV\df_soc.csv')
    return  df_socc

In [11]:
def merge():
    ''' TWO WAY MERGES
    
        STARBET / ADMIRALBET '''
    try:
        df_star, df_adm = pd.read_csv('CSV\df_star.csv') , pd.read_csv('CSV\df_adm.csv')


        teams_star = df_star.teams.tolist()

        df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_adm = df_adm[df_adm.score>60]

        df_star_adm = pd.merge(df_adm, df_star, left_on='teams_matched', right_on='teams', how='left')
        df_star_adm.to_csv('CSV\TwoWay\Live_star_adm.csv')
    except:
        print('Something went wrong. CHECK: STARBET AND ADRMIRALBET live PAGES')
        pass

    '''          ADMIRALBET / SOCCERBET ''' 
    try:
        df_adm, df_soc = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_soc.csv')

        teams_adm = df_adm.teams.tolist()
        #teams_soc = df_soc.teams.tolist()

        df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_adm, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc = df_soc[df_soc.score>50]

        df_soc_adm = pd.merge(df_soc, df_adm, left_on='teams_matched', right_on='teams', how='left')
        df_soc_adm.to_csv('CSV\TwoWay\Live_soc_adm.csv')
    except:
        print('Something went wrong. CHECK: SOCCERBET AND ADRMIRALBET live PAGES')
        pass

    '''         STARBET / SOCCERBET ''' 
    try:
        df_star, df_soc = pd.read_csv('CSV\df_star.csv'), pd.read_csv('CSV\df_soc.csv')

        teams_star = df_star.teams.tolist()
        #teams_soc = df_soc.teams.tolist()

        df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc = df_soc[df_soc.score>50]

        df_soc_star = pd.merge(df_soc, df_star, left_on='teams_matched', right_on='teams', how='left')
        df_soc_star.to_csv('CSV\TwoWay\Live_soc_star.csv') 
    except:
        print('Something went wrong. CHECK: STARBET AND SOCCERBET live PAGES')

    '''          MAXBET / SOCCERBET '''
    try:
        df_soc, df_max = pd.read_csv('CSV\df_soc.csv'), pd.read_csv('CSV\df_max.csv')

        teams_max = df_max.teams.tolist()
        #teams_soc = df_soc.teams.tolist()

        df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc = df_soc[df_soc.score>59]

        df_soc_max = pd.merge(df_soc, df_max, left_on='teams_matched', right_on='teams', how='left')
        df_soc_max.to_csv('CSV\TwoWay\Live_soc_max.csv')
    except:
        print('Something went wrong. CHECK: STARBET AND MAXBET live PAGES')
        pass
    '''          MAXBET / ADMIRAL ''' 
    try:
        df_adm, df_max = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_max.csv')

        #teams_adm = df_adm.teams.tolist()
        teams_max = df_max.teams.tolist()

        df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
        df_adm = df_adm[df_adm.score>59]

        df_adm_max = pd.merge(df_adm, df_max, left_on='teams_matched', right_on='teams', how='left')
        df_adm_max.to_csv('CSV\TwoWay\Live_adm_max.csv')
    except:
        print('Something went wrong. CHECK: MAXBET AND ADRMIRALBET live PAGES')
        pass

    '''          MAXBET / STARBET ''' 
    try:
        df_max, df_star =  pd.read_csv('CSV\df_max.csv'), pd.read_csv('CSV\df_star.csv')
        
        teams_star = df_star.teams.tolist()

        df_max[['teams_matched', 'score']] = df_max.teams.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_max = df_max[df_max.score>59]

        df_max_star = pd.merge(df_max,df_star, left_on='teams_matched', right_on='teams', how='left')
        df_max_star.to_csv('CSV\TwoWay\Live_max_star.csv')
    except:
        print('Something went wrong. CHECK: STARBET AND MAXBET live PAGES')
        pass

    ''' 3WAY MERGES''' 


    ''' AD / ST / SC '''
    try:
    # loading data
        df_soc_adm = pd.read_csv('CSV\TwoWay\Live_soc_adm.csv')
        df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
        df_star = pd.read_csv('CSV\df_star.csv')
        #Start Merging process
        teams_star = df_star.teams.tolist()
        df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
        #The last merging process
        df_soc_adm_star = pd.merge(df_soc_adm, df_star, left_on='teams_matched2', right_on='teams', how='left')
        df_soc_adm_star.to_csv('CSV\ThreeWay\Live_soc_adm_star.csv')
    except:
        pass


    ''' AD / SC / MAX '''
    try:
        # loading data
        df_soc_adm = pd.read_csv('CSV\TwoWay\Live_soc_adm.csv')
        df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
        df_max = pd.read_csv('CSV\df_max.csv')
        #Start Merging process
        teams_max = df_max.teams.tolist()
        df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
        #The last merging process
        df_soc_adm_max = pd.merge(df_soc_adm, df_max, left_on='teams_matched2', right_on='teams', how='left')
        df_soc_adm_max.to_csv('CSV\ThreeWay\Live_soc_adm_max.csv')
    except:
        pass


    ''' SC / ST / MAX '''
    try:
        df_soc_star = pd.read_csv('CSV\TwoWay\Live_soc_star.csv')
        df_soc_star.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)

        teams_max = df_max.teams.tolist()
        df_soc_star[['teams_matched2', 'score']] = df_soc_star.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_soc_star = df_soc_star[df_soc_star.score>59] #This drops all the teams names that don't match among sites
        #The last merging process
        df_soc_star_max = pd.merge(df_soc_star, df_max, left_on='teams_matched2', right_on='teams', how='left')
        df_soc_star_max.to_csv('CSV\ThreeWay\Live_soc_star_max.csv')
    except:
        pass

In [23]:
def final_function(minutos):
    # TIME STUFF
    target = pd.datetime.now() + pd.to_timedelta(minutos, unit='min')
    target_minute = target.minute
    target_hour = target.hour
    now = pd.datetime.now()

    i=0
    #THREADS
    # Put everything to work
    while target>now:

        now = pd.datetime.now()
        
        starbet  = threading.Thread(target=starbet_live)
        maxbet   = threading.Thread(target=maxbet_live)
        admiralbet  = threading.Thread(target=admiral_live)
        soccerbet= threading.Thread(target=soccerbet_live)
        
        clear_output()
        i+=1
        print(f'Will work till: {target_hour}hs and {target_minute}mins')
        print(f'Iteration number: {i}')
        print('-------')
        print('Functions running')

        maxbet.start()
        admiralbet.start()
        soccerbet.start()
        starbet.start()

        admiralbet.join()
        soccerbet.join()
        starbet.join()
        maxbet.join()
        

        print('Scrapers done')
        print('-------')
        print('Merging start...')
        merge()
        print('-------')
        print('Merging finished! All csv files have been updated')
        time.sleep(3)
    
    #print(f'iteration {i}/{tiempo}')

For __enable__/__disable__ some of the pages, you should go inside the `final_function(minutos):` and find the page you want to enable/disable and __ADD__ a ___#___ before `page`.start() __and__ `page`.join() you want to enable/disable. 
 For example:
* maxbet: #maxbet.start() + #maxbet.join() 
    



In [22]:
final_function(30)

Will work till: 13hs and 56mins
Iteration number: 122


  0%|          | 0/80 [00:00<?, ?it/s]

  2%|▎         | 2/80 [00:00<00:05, 14.48it/s]

-------
Functions running


100%|██████████| 80/80 [00:07<00:00, 10.41it/s]


-------
Scrapers done
-------
Merging start
-------
Merging finished! All csv files have been updated
